In [1]:
import pandas as pd
import requests
import json
import getpass

## Clean 'country_codes_OECD.csv' file obtained from WeatherBit

Import 'country_codes_OECD.csv' file that contains all the country codes from WeatherBit

In [2]:
country_codes_OECD = pd.read_csv('..\csv_files\country_codes_OECD.csv', engine='python')

In [3]:
country_codes_OECD.head()

,code,country,Unnamed: 2,Unnamed: 3
0,ABW,Aruba,NaN,NaN
1,AFG,Afghanistan,NaN,NaN
2,AFRI,Africa,NaN,NaN
3,AGO,Angola,NaN,NaN
4,AIA,Anguilla,NaN,NaN


Remove whitespaces from column names

In [4]:
country_codes_OECD.columns = country_codes_OECD.columns.str.replace(' ','')

Look at shape of the dataframe

In [5]:
country_codes_OECD.shape

(234, 4)

Investigate null values

In [6]:
country_codes_OECD.isna().sum()

code           0
country        0
Unnamed:2    234
Unnamed:3    233
dtype: int64

Investigate the solitary non-null value in 'Unnamed:3' column

In [7]:
country_codes_OECD.groupby(by=["Unnamed:3"]).sum()

,Unnamed:2
Unnamed:3,
Ukraine,0.0


Columns 'Unnamed:2' & 'Unnamed:3' can be dropped

In [8]:
country_codes_OECD = country_codes_OECD.drop(['Unnamed:2'], axis=1)
country_codes_OECD = country_codes_OECD.drop(['Unnamed:3'], axis=1)

In [9]:
country_codes_OECD['code']

0       ABW
1       AFG
2      AFRI
3       AGO
4       AIA
       ... 
229     WSM
230     YEM
231     ZAF
232     ZMB
233     ZWE
Name: code, Length: 234, dtype: object

In [10]:
code_list = country_codes_OECD['code'].tolist()
print(code_list)

['ABW', 'AFG', 'AFRI', 'AGO', 'AIA', 'ALB', 'AND', 'ANT', 'ARE', 'ARG', 'ASIA', 'ASM', 'ATG', 'AUS', 'AUT', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CCK', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CSFR', 'CSFR-CZE', 'CSFR-SVK', 'CUB', 'CYM', 'CYP', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESH', 'ESP', 'ETH', 'EURO', 'FIN', 'FJI', 'FLK', 'FRA', 'FSM', 'FYUG', 'FYUG-BIH', 'FYUG-HRV', 'FYUG-MKD', 'FYUG-SVN', 'FYUG-YUG', 'GAB', 'GBR', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HTI', 'HUN', 'IDN', 'IND', 'IOT', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KEN', 'KHM', 'KIR', 'KNA', 'KOREA-NO', 'KOREA-NS', 'KOREA-SO', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LUX', 'MAC', 'MAR', 'MCO', 'MDG', 'MDV', 'MEX', 'MHL', 'MLI', 'MLT', 'MMR', 'MNG', 'MNP', 'MOZ',

In [11]:
for code in code_list:
    if len(code) > 3:
        print(code)

AFRI
ASIA
CSFR
CSFR-CZE
CSFR-SVK
EURO
FYUG
FYUG-BIH
FYUG-HRV
FYUG-MKD
FYUG-SVN
FYUG-YUG
KOREA-NO
KOREA-NS
KOREA-SO
NOAM
OCEA
SCAC
USSR
USSR-ARM
USSR-AZE
USSR-BLR
USSR-EST
USSR-GEO
USSR-KAZ
USSR-KGZ
USSR-LTU
USSR-LVA
USSR-MDA
USSR-RUS
USSR-TJK
USSR-TKM
USSR-UKR
USSR-UZB


Codes that are exactly 4 characters long are continents or former countries, so they can be dropped

In [12]:
four_codes = country_codes_OECD.loc[country_codes_OECD['code'].str.len() == 4].index
country_codes_OECD = country_codes_OECD.drop(four_codes)

'KOREA-SO' can be deleted, 'KOREA-NO' can be changed to 'PRK', 'KOREA-NS' can be changed to 'KOR'                                                              

In [13]:
row_drop = country_codes_OECD.loc[country_codes_OECD['code'] == 'KOREA-SO'].index
country_codes_OECD = country_codes_OECD.drop(row_drop)
country_codes_OECD = country_codes_OECD.replace(to_replace={'KOREA-NO': 'PRK', 'KOREA-NS': 'KOR'})

Reset index

In [14]:
country_codes_OECD.reset_index(drop=True, inplace=True)

All remaining country codes can have their prefix removed

In [15]:
prefix_codes = country_codes_OECD.loc[country_codes_OECD['code'].str.len() == 8].index
prefix_codes

Int64Index([ 44,  45,  66,  67,  68,  69,  70, 197, 198, 199, 200, 201, 202,
            203, 204, 205, 206, 207, 208, 209, 210, 211],
           dtype='int64')

In [16]:
country_codes_OECD["code"].iloc[prefix_codes] = country_codes_OECD["code"].iloc[prefix_codes].str.slice(start=5)

In [17]:
country_codes_OECD["code"].tolist()

['ABW',
 'AFG',
 'AGO',
 'AIA',
 'ALB',
 'AND',
 'ANT',
 'ARE',
 'ARG',
 'ASM',
 'ATG',
 'AUS',
 'AUT',
 'BDI',
 'BEL',
 'BEN',
 'BFA',
 'BGD',
 'BGR',
 'BHR',
 'BHS',
 'BLZ',
 'BMU',
 'BOL',
 'BRA',
 'BRB',
 'BRN',
 'BTN',
 'BWA',
 'CAF',
 'CAN',
 'CCK',
 'CHE',
 'CHL',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COG',
 'COK',
 'COL',
 'COM',
 'CPV',
 'CRI',
 'CZE',
 'SVK',
 'CUB',
 'CYM',
 'CYP',
 'DEU',
 'DJI',
 'DMA',
 'DNK',
 'DOM',
 'DZA',
 'ECU',
 'EGY',
 'ERI',
 'ESH',
 'ESP',
 'ETH',
 'FIN',
 'FJI',
 'FLK',
 'FRA',
 'FSM',
 'BIH',
 'HRV',
 'MKD',
 'SVN',
 'YUG',
 'GAB',
 'GBR',
 'GHA',
 'GIB',
 'GIN',
 'GMB',
 'GNB',
 'GNQ',
 'GRC',
 'GRD',
 'GTM',
 'GUM',
 'GUY',
 'HKG',
 'HND',
 'HTI',
 'HUN',
 'IDN',
 'IND',
 'IOT',
 'IRL',
 'IRN',
 'IRQ',
 'ISL',
 'ISR',
 'ITA',
 'JAM',
 'JOR',
 'JPN',
 'KEN',
 'KHM',
 'KIR',
 'KNA',
 'PRK',
 'KOR',
 'KWT',
 'LAO',
 'LBN',
 'LBR',
 'LBY',
 'LCA',
 'LIE',
 'LKA',
 'LSO',
 'LUX',
 'MAC',
 'MAR',
 'MCO',
 'MDG',
 'MDV',
 'MEX',
 'MHL',
 'MLI',
 'MLT',


Export data to CSV file

In [18]:
country_codes_OECD = country_codes_OECD.to_csv('..\csv_files\country_codes_final.csv', index=False)

## Import and clean 'protected_areas.csv' file

In [22]:
protected_areas = pd.read_csv('..\csv_files\protected_areas_raw.csv')

In [23]:
protected_areas.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,PROTECTEDAREA,TERREST,PC_LANDAREA,A,2015,18.14,NaN
1,AUS,PROTECTEDAREA,TERREST,PC_LANDAREA,A,2016,18.85,NaN
2,AUS,PROTECTEDAREA,TERREST,PC_LANDAREA,A,2017,19.10,NaN
3,AUS,PROTECTEDAREA,TERREST,PC_LANDAREA,A,2018,19.20,NaN
4,AUS,PROTECTEDAREA,TERREST,PC_LANDAREA,A,2019,19.20,NaN


Investigate null values

In [24]:
protected_areas.isna().sum()

LOCATION        0
INDICATOR       0
SUBJECT         0
MEASURE         0
FREQUENCY       0
TIME            0
Value           0
Flag Codes    438
dtype: int64

Delete 'Flag Codes' column

In [25]:
protected_areas.drop('Flag Codes', axis=1, inplace=True)

Check for columns that have only one unique value, and drop columns that do

In [26]:
for col in protected_areas.columns:
    if len(protected_areas[col].unique()) == 1:
        protected_areas.drop(col, axis=1, inplace=True)

In [27]:
protected_areas

,LOCATION,TIME,Value
0,AUS,2015,18.14
1,AUS,2016,18.85
2,AUS,2017,19.10
3,AUS,2018,19.20
4,AUS,2019,19.20
...,...,...,...
433,URY,2016,3.48
434,URY,2017,3.48
435,URY,2018,3.48
436,URY,2019,3.62


Use pivot function to create new columns for each year in the dataframe

In [28]:
protected_areas = protected_areas.pivot(index='LOCATION', columns='TIME')
protected_areas.head()

Value                                   
TIME       2015   2016   2017   2018   2019   2020
LOCATION                                          
ARG        8.27   8.29   8.33   8.39   8.39   8.39
ARM       24.11  24.11  24.11  24.11  24.11  24.11
AUS       18.14  18.85  19.10  19.20  19.20  19.20
AUT       27.97  28.45  28.45  28.54  28.54  28.54
AZE        5.63   5.63   5.63   5.93   5.93   5.93

Reintroduce index, moving country codes into the dataframe

In [29]:
protected_areas.reset_index(inplace=True)

In [30]:
protected_areas.head()

LOCATION  Value                                   
TIME            2015   2016   2017   2018   2019   2020
0         ARG   8.27   8.29   8.33   8.39   8.39   8.39
1         ARM  24.11  24.11  24.11  24.11  24.11  24.11
2         AUS  18.14  18.85  19.10  19.20  19.20  19.20
3         AUT  27.97  28.45  28.45  28.54  28.54  28.54
4         AZE   5.63   5.63   5.63   5.93   5.93   5.93

In [31]:
protected_areas.columns

MultiIndex([('LOCATION',   ''),
            (   'Value', 2015),
            (   'Value', 2016),
            (   'Value', 2017),
            (   'Value', 2018),
            (   'Value', 2019),
            (   'Value', 2020)],
           names=[None, 'TIME'])

Remove multiindex

In [32]:
[x for x in protected_areas.columns]

[('LOCATION', ''),
 ('Value', 2015),
 ('Value', 2016),
 ('Value', 2017),
 ('Value', 2018),
 ('Value', 2019),
 ('Value', 2020)]

In [33]:
[x[1] for x in protected_areas.columns]

['', 2015, 2016, 2017, 2018, 2019, 2020]

In [34]:
protected_areas.columns = [x[1] for x in protected_areas.columns]

Rename column containing country codes

In [35]:
protected_areas.rename(columns={ protected_areas.columns[0]: "country_code" }, inplace = True)

In [36]:
protected_areas.head()

,country_code,2015,2016,2017,2018,2019,2020
0,ARG,8.27,8.29,8.33,8.39,8.39,8.39
1,ARM,24.11,24.11,24.11,24.11,24.11,24.11
2,AUS,18.14,18.85,19.10,19.20,19.20,19.20
3,AUT,27.97,28.45,28.45,28.54,28.54,28.54
4,AZE,5.63,5.63,5.63,5.93,5.93,5.93


In [37]:
protected_areas = protected_areas.to_csv('..\csv_files\protected_areas.csv', index=False)